In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from fuzzywuzzy import fuzz


In [4]:
peliculas = pd.read_csv("C:/Users/Fede/Documents/henry/Labs/Fanny/peliculas.csv")

In [5]:
# Eliminar filas con valores faltantes en las columnas relevantes para el análisis
peliculas.dropna(subset=['genres', 'belongs_to_collection', 'original_language', 'budget'], inplace=True)

In [6]:
# Limpiar y estandarizar el texto para facilitar la búsqueda
peliculas['title'] = peliculas['title'].str.lower().str.strip()

peliculas['combined_features'] = (
    peliculas['genres'].astype(str) + ' ' +
    peliculas['belongs_to_collection'].astype(str) + ' ' +
    peliculas['original_language'].astype(str) + ' ' +
    peliculas['budget'].astype(str)
)

In [7]:
# Crear la matriz de características TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(peliculas['combined_features'])

In [15]:
# Calcular la similitud del coseno utilizando el kernel lineal
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def recomendacion(titulo):
    # Convertir el título ingresado a minúsculas y eliminar espacios en blanco
    titulo = titulo.lower().strip()

    # Realizar búsqueda difusa para encontrar el título más similar en el DataFrame
    match_scores = peliculas['title'].apply(lambda x: fuzz.partial_ratio(x.lower().strip(), titulo))
    best_match_index = match_scores.idxmax()

    # Obtener el índice de la película correspondiente al título más similar
    index = best_match_index

    # Calcular la similitud de la película con el resto de películas
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener los índices de las 5 películas más similares (excluyendo la película consultada)
    similar_movies_indices = [i[0] for i in sim_scores[1:6]]

    # Obtener los nombres de las películas recomendadas
    recommended_movies = peliculas['title'].iloc[similar_movies_indices].to_list()

    return recommended_movies


In [17]:
titulo_consulta = "Toy Story"

# Obtener las películas recomendadas utilizando la función recomendacion
peliculas_recomendadas = recomendacion(titulo_consulta)

# Imprimir las películas recomendadas
print("Película consultada:", titulo_consulta)
print("Películas recomendadas:")
for idx, pelicula in enumerate(peliculas_recomendadas, start=1):
    print(f"{idx}. {pelicula}")

Película consultada: Toy Story
Películas recomendadas:
1. toy story 3
2. toy story 2
3. a christmas story
4. the spongebob squarepants movie
5. hoodwinked too! hood vs. evil
